In [ ]:
from matplotlib import pyplot as plt
import pandas as pd;
import numpy as np;
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Wij hebben de data vergeleken en zien of via de fullstack developers de gemiddelde logins per uur per weekdag hetzelfde waren

data = pd.read_csv("dataset_aangepast_nieuw.csv", sep=";")
print(data)
data['Datum en Tijd'] = pd.to_datetime(data['Datum en Tijd'], format="%d/%m/%Y %H:%M")
# Maak een lege dictionary om de resultaten op te slaan
weekly_data = {}

# Haal de unieke netwerken op
networks = data['Locatie'].unique()

# Verwijder alle rijen met NaN waarden
data = data.dropna()

data = data[data['Aantal Logins'] != 0.0]

# Verwijder alle rijen waar de 'Netwerk' of 'AantalLogins' kolom NaN is
data = data.dropna(subset=['Locatie', 'Aantal Logins'])

# Loop over elk netwerk
for network in networks:
    # Filter de data voor het specifieke netwerk
    network_data = data[data['Locatie'] == network].dropna()

    # Loop over elke dag van de week (waarbij maandag = 0 en zondag = 6)
    for day in range(5):
        # Filter de data voor de specifieke dag
        day_data = network_data[network_data['Datum en Tijd'].dt.dayofweek == day].dropna()
        day_data['hour'] = day_data['Datum en Tijd'].dt.hour
        filtered_data = day_data[(day_data['hour'] >= 8) & (day_data['hour'] <= 18)].dropna()

        # Bereken het minimum aantal logins tijdens de nacht
        night_data = day_data[((day_data['hour'] >= 19) & (day_data['hour'] <= 23)) | ((day_data['hour'] >= 0) & (day_data['hour'] <= 7))].dropna()
        min_night_logins = night_data['Aantal Logins'].min()

        # Verminder het aantal logins van 8-18 met het minimum aantal logins van 19-7
        filtered_data['Aantal Logins'] = filtered_data['Aantal Logins'] - min_night_logins

        # Groepeer de gefilterde data per uur en bereken het gemiddelde aantal logins
        filtered_hourly_mean = filtered_data.groupby('hour')['Aantal Logins'].mean().dropna()

        # Voeg de resultaten toe aan de dictionary
        weekly_data[(network, day)] = round(filtered_hourly_mean,0)

print(weekly_data)

# Print de resultaten
for (network, day), hourly_mean in weekly_data.items():
    print(f"Network {network}, Day {day}:")
    print(hourly_mean)


# Groepeer de data per weekdag en netwerk en bereken het gemiddelde aantal logins
weekly_data = data.groupby([data['Datum en Tijd'].dt.weekday, 'Locatie'])['Aantal Logins'].mean().reset_index()

# Voeg een uur kolom toe aan de data
data['Hour'] = data['Datum en Tijd'].dt.hour

# Groepeer de data per uur en netwerk en bereken het gemiddelde aantal logins
hourly_data = data.groupby(['Hour', 'Locatie'])['Aantal Logins'].mean().reset_index()

def get_color(logins):
    if logins > 10:  # Dit is een voorbeeld, vervang 100 door de drempel voor "veel" logins
        return 'red'
    elif logins > 5:  # Dit is een voorbeeld, vervang 50 door de drempel voor "medium" logins
        return 'orange'
    else:
        return 'green'

# Filter de data voor de uren van 8 tot 18 en de dagen van maandag tot vrijdag
filtered_data = data[(data['Hour'] >= 8) & (data['Hour'] <= 18) & (data['Datum en Tijd'].dt.dayofweek >= 0) & (data['Datum en Tijd'].dt.dayofweek <= 5)]

# Groepeer de gefilterde data per uur en netwerk en bereken het gemiddelde aantal logins
filtered_hourly_data = filtered_data.groupby(['Hour', 'Locatie'])['Aantal Logins'].mean().reset_index()

# Loop over elk netwerk
# for network in filtered_hourly_data['Locatie'].unique():
#     # Filter de data voor het specifieke netwerk
#     network_data = filtered_hourly_data[filtered_hourly_data['Locatie'] == network]
    
#     # Maak een figuur en een as
#     fig, ax = plt.subplots(figsize=(10, 6))
    
#     # Plot de data als een staafdiagram met kleurcodes
#     ax.bar(network_data['Hour'], network_data['Aantal Logins'], color=network_data['Aantal Logins'].apply(get_color), label=network)
    
#     # Stel de labels en de titel in
#     ax.set_xlabel('Hour')
#     ax.set_ylabel('Average Logins')
#     ax.set_title(f'Average Logins per Hour for {network}')
#     ax.legend()
    
#     # Toon de plot
#     plt.show()

# Filter de dataset voor de dagen maandag tot en met vrijdag en de uren tussen 8:00 en 18:00
weekday_filtered_data = data[(data['Datum en Tijd'].dt.hour >= 8) & (data['Datum en Tijd'].dt.hour <= 18) & (data['Datum en Tijd'].dt.weekday <= 5)]

# Voeg kolommen toe voor weekdag en uur
weekday_filtered_data['Weekdag'] = weekday_filtered_data['Datum en Tijd'].dt.day_name()
weekday_filtered_data['Uur'] = weekday_filtered_data['Datum en Tijd'].dt.hour

hourly_average_logins = weekday_filtered_data.groupby(['Locatie', 'Weekdag', 'Uur'])['Aantal Logins'].mean().unstack().unstack()

# Verwijder rijen met NaN waarden
hourly_average_logins = hourly_average_logins.dropna()
# Bereken het gemiddelde aantal logins per uur per weekdag voor elk netwerk met behoud van de multi-level index
hourly_average_df = weekday_filtered_data.groupby(['Locatie', 'Weekdag', 'Uur'])['Aantal Logins'].mean().reset_index()

# Maak een pivot tabel voor betere weergave en toegang
hourly_average_pivot_df = hourly_average_df.pivot_table(index=['Locatie', 'Weekdag'], columns='Uur', values='Aantal Logins')
# Creëer afzonderlijke DataFrames voor elke weekdag
monday_df = hourly_average_df[hourly_average_df['Weekdag'] == 'Monday'].pivot_table(index='Locatie', columns='Uur', values='Aantal Logins')
tuesday_df = hourly_average_df[hourly_average_df['Weekdag'] == 'Tuesday'].pivot_table(index='Locatie', columns='Uur', values='Aantal Logins')
wednesday_df = hourly_average_df[hourly_average_df['Weekdag'] == 'Wednesday'].pivot_table(index='Locatie', columns='Uur', values='Aantal Logins')
thursday_df = hourly_average_df[hourly_average_df['Weekdag'] == 'Thursday'].pivot_table(index='Locatie', columns='Uur', values='Aantal Logins')
friday_df = hourly_average_df[hourly_average_df['Weekdag'] == 'Friday'].pivot_table(index='Locatie', columns='Uur', values='Aantal Logins')
saturday_df = hourly_average_df[hourly_average_df['Weekdag'] == 'Saturday'].pivot_table(index='Locatie', columns='Uur', values='Aantal Logins')


# Functie om bar charts te maken voor elke weekdag
def plot_hourly_logins(df, weekday):
    ax = df.plot(kind='bar', figsize=(14, 8), legend=True, title=f'Gemiddelde logins per uur op {weekday}')
    ax.set_xlabel('Locatie')
    ax.set_ylabel('Gemiddeld aantal logins')
    ax.legend(title='Uur')
    plt.xticks(rotation=45)
    plt.show()

# Plot de bar chart voor maandag
plot_hourly_logins(monday_df, "Maandag")
plot_hourly_logins(tuesday_df, "Dinsdag")
plot_hourly_logins(wednesday_df, "Woensdag")
plot_hourly_logins(thursday_df, "Donderdag")
plot_hourly_logins(friday_df, "Vrijdag")
plot_hourly_logins(saturday_df, "Zaterdag")



# Code uitleg

Hieronder zorgen wij ervoor dat wij onze dataset gaan aanpassen zodat het in de database kan opgeladen worden en zodat wij hiermee onze programma's kunnen runnen.
Wij filteren de lege rijen / de Not a Number waarden, wij probeerden ook alle 0 values te verwijderen maar dit lukt niet helemaal
Wij hebben er ook voor gezorgd dat de aantal logins gefilterd wordt zodat het de meest accurate logins heeft.

In [2]:
import pandas as pd

# 1. Lees de originele dataset in
data = pd.read_csv("dataset_aangepast_nieuw.csv", sep=";")

# 2. Verwijder alle rijen met NaN waarden
data = data.dropna()

# 3. Verwijder alle rijen waar 'Aantal Logins' gelijk is aan 0.0
data = data[data['Aantal Logins'] != 0]

# 4. Converteer de 'Datum en Tijd' kolom naar een datetime object
data['Datum en Tijd'] = pd.to_datetime(data['Datum en Tijd'], format="%d/%m/%Y %H:%M")


# 6. Filter de data voor de uren van 8 tot 18 en bereken het minimum aantal logins tijdens de nacht (0-7 en 19-23)
night_data = data[((data['Datum en Tijd'].dt.hour >= 19) & (data['Datum en Tijd'].dt.hour <= 23)) | ((data['Datum en Tijd'].dt.hour >= 0) & (data['Datum en Tijd'].dt.hour <= 7))]
min_night_logins = night_data['Aantal Logins'].min()

day_data = data[(data['Datum en Tijd'].dt.hour >= 8) & (data['Datum en Tijd'].dt.hour <= 18)]
day_data['Aantal Logins'] = day_data['Aantal Logins'] - min_night_logins

# 7. Trek het minimum aantal logins van de nacht af van de logins van 8-18
data.loc[(data['Datum en Tijd'].dt.hour >= 8) & (data['Datum en Tijd'].dt.hour <= 18), 'Aantal Logins'] = day_data['Aantal Logins']

# 8. Sla de nieuwe dataset op

data.to_csv("NieuweDefinitieveDataset.csv", sep=";", index=False)

C:\Users\Dawa Lama\AppData\Local\Temp\ipykernel_5636\2199233041.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data['Aantal Logins'] = day_data['Aantal Logins'] - min_night_logins
